In [ ]:
      # "orientation": [
        #     0.000014337465718738874,
        #     -0.04198668285846087,
        #     0.9991181495114061,
        #     -0.00020389674746602128
        # ]
        # {}


In [7]:
from space import Pose, Vector3, Quaternion, Euler
import aiohttp
import asyncio
import socketio
import logging
sio = socketio.AsyncClient(logger=False, engineio_logger=False)
await sio.connect('http://192.168.2.13:8081')

rotational_speed = 0.0
linear_speed = 0.0

max_rotation_speed = 0.2617994 # radians/s
max_translation_speed = 0.25; # 250mm/s



async def setTranslationSpeed(arg0: float) -> None:
    if arg0 > max_translation_speed:
        print("Speed too high, setting to max speed.")
        arg0 = max_translation_speed
    async with aiohttp.ClientSession() as session:
        async with session.patch("http://192.168.2.13:8081/api/cartesian", json={"linearVelocity": arg0}) as response:
            print(await response.text())
            await sio.emit("linearveltrigger", {"data": True})


async def setRotationalSpeed(arg0: float) -> None:
    if arg0 > max_rotation_speed:
        print("Speed too high, setting to max speed.")
        arg0 = max_rotation_speed
    async with aiohttp.ClientSession() as session:
        async with session.patch("http://192.168.2.13:8081/api/cartesian", json={"rotationSpeed": arg0}) as response:
            print(await response.text())
            await sio.emit("rotationalveltrigger", {"data": True})

customC = 60 

def deg2rad(deg):
    return deg * 0.0174533

async def move(v : Vector3, along_normal: bool = False) -> None:
    newA = 0
    newB = 0
    newC = 0
    if along_normal:
        a = 0
        b = 0
        c = 0 + deg2rad(customC)
        euler = Euler(a, b, c)
        rotMatrix = euler.to_matrix()
        movementVector = rotMatrix @ v
    else:
        movementVector = v

    for i in range(10):
        await start_movement_slider(
            movementVector.x,
            movementVector.y,
            movementVector.z,
            newA,
            newB,
            newC
        )
        await asyncio.sleep(0.1)
    await stop_movement_slider(0, 0, 0, 0, 0, 0)
    

async def setRotSpeedDegS(arg0):
    rotational_speed = arg0 / 180 * max_rotation_speed;
    print(f"Setting rotational speed to {rotational_speed} rad/s or {arg0} deg/s")
    await setRotationalSpeed(rotational_speed);

async def setTranslationSpeedMMs(arg0):
    linear_speed = arg0 / 1000 * max_translation_speed;
    print(f"Setting translation speed to {linear_speed} m/s or {arg0} mm/s")
    await setTranslationSpeed(linear_speed);


async def start_movement_slider(q0, q1, q2, q3, q4, q5):
    data = {
        'q0': q0,
        'q1': q1,
        'q2': q2,
        'q3': q3,
        'q4': q4,
        'q5': q5,
        'status': True,
        'joint': False,
        'cartesian': True,
        'freedrive': False,
        'button': False,
        'slider': True,
        'goto': False,
        'threeD': False,
        'reference': "Base",
        'absrel': "Absolute",
    }
    await sio.emit('CartesianSlider', data)

async def stop_movement_slider(q0, q1, q2, q3, q4, q5):
    data = {
        'q0': q0,
        'q1': q1,
        'q2': q2,
        'q3': q3,
        'q4': q4,
        'q5': q5,
        'status': False,
        'joint': False,
        'cartesian': True,
        'freedrive': False,
        'button': False,
        'slider': True,
        'goto': False,
        'threeD': False,
        'reference': "Base",
        'absrel': "Absolute",
    }
    await sio.emit('CartesianSlider', data)


await setTranslationSpeedMMs(10)
await move(Vector3(1, 0, 0), True)


Setting translation speed to 0.0025 m/s or 10 mm/s
{"RobotSettings":{"workspaceMin":{"X":-3,"Y":-3,"Z":-3},"workspaceMax":{"X":3,"Y":3,"Z":3},"velocityJog":{"Vx":3.6,"Vy":1,"Vz":1,"Va":1.570796,"Vb":0.523598,"Vc":0.523598}},"SelectedTool":{"Name":"NoTool","id":"661a8224651be61e742d8a79"},"SafetySettings":{"cartMaxVel":{"Vx":0,"Vy":0,"Vz":0,"Va":0,"Vb":0,"Vc":0},"torqueMaxTCP":{"mx":0,"my":0,"mz":0},"forceMaxTCP":{"fx":0,"fy":0,"fz":0}},"line":100,"linearVelocity":0.0025,"runningmode":"Real","collision":true,"collisionDeactivated":false,"isHmiActive":false,"incOnOff":false,"OnOff":false,"incrementMovement":0,"rotationSpeed":0.1745329,"_id":"661a8225651be61e742d8a8a","__v":0}


In [3]:
from lara import Lara
from space import Pose, Vector3, Quaternion, Euler
import aiohttp
import asyncio
import socketio
import logging

lara = Lara()
await lara.connect_socket()
await lara.setTranslationSpeedMMs(250)
await lara.move(Vector3(-1, 0, 0), True)


[2025-01-06 15:05:31][neurapy_logger][WARNING] : Current client version is not compatiable with the version of the server running on the robot. Some of the functionlities specified in the documentation might not work in the intended way. Please upgrade to the correct version .Client Version : {VERSION},Server Version : {self.version} :(robot.py:176)
Setting translation speed to 0.0625 m/s or 250 mm/s
connection established
{"RobotSettings":{"workspaceMin":{"X":-3,"Y":-3,"Z":-3},"workspaceMax":{"X":3,"Y":3,"Z":3},"velocityJog":{"Vx":3.6,"Vy":1,"Vz":1,"Va":1.570796,"Vb":0.523598,"Vc":0.523598}},"SelectedTool":{"Name":"NoTool","id":"661a8224651be61e742d8a79"},"SafetySettings":{"cartMaxVel":{"Vx":0,"Vy":0,"Vz":0,"Va":0,"Vb":0,"Vc":0},"torqueMaxTCP":{"mx":0,"my":0,"mz":0},"forceMaxTCP":{"fx":0,"fy":0,"fz":0}},"line":100,"linearVelocity":0.0625,"runningmode":"Real","collision":true,"collisionDeactivated":false,"isHmiActive":false,"incOnOff":false,"OnOff":false,"incrementMovement":0,"rotation

C:\Users\nxp84358\AppData\Local\Temp\2\ipykernel_11672\2856218647.py:9: RuntimeWarning: coroutine 'Lara.connect_socket' was never awaited
  lara.connect_socket()


Exception in thread Thread-6 (notify_diagnostics):
Traceback (most recent call last):
  File "c:\Users\nxp84358\Documents\GitHub\lara-control\python\neura\neurapy\robot.py", line 76, in wrapped_function
    sock.connect(address)
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\nxp84358\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "C:\Users\nxp84358\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\nxp84358\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\nxp84358\Documents\GitHub\lara-control\python\neura\neurapy\robot.py", line 185, in notify_diagnosti

In [10]:
import cv2
import numpy as np
import os
import glob
import time
from IPython.display import clear_output
import json

tagSize = 0.2

corner = [
    {
        "x": 445.92,
        "y": 340.88
    },
    {
        "x": 445.86,
        "y": 449.92
    },
    {
        "x": 553.93,
        "y": 449.86
    },
    {
        "x": 553.84,
        "y": 340.89
    }
]

corner = np.array([(point['x'], point['y']) for point in corner], dtype=np.float64)

objectPoints = np.array([
            [-tagSize / 2, -tagSize / 2, 0],
            [tagSize / 2, -tagSize / 2, 0],
            [tagSize / 2, tagSize / 2, 0],
            [-tagSize / 2, tagSize / 2, 0],
        ], dtype=np.float64)

calibration_data = {
        "mtx": [
            [151.13836173368213, 0.0, 249.16869615047108],
            [0.0, 151.07555317759122, 247.0790049731641],
            [0.0, 0.0, 1.0],
        ],
        "dist": [
            [
                -0.0004072754849863926, 0.0008449432688030175,
                0.0004268710978072279, 2.0400336739956796e-6,
                -0.00011737345044632326,
            ],
        ],
    }

r = cv2.solvePnP(objectPoints, corner, np.array(calibration_data['mtx']), np.array(calibration_data['dist']))
rvec = r[1]
tvec = r[2]
print(tvec)

#result
# [[0.46182605]
#  [0.27265001]
#  [0.27948932]]

[[0.46182605]
 [0.27265001]
 [0.27948932]]
